In [58]:
import sys
sys.path.append('../..')
from sqlalchemy import create_engine, MetaData, Table, text

import asyncio
import httpx
from datetime import datetime, timedelta
import time
import pandas as pd

from config import POSTGRES_UTEA

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')
VIAJES = 500
PROMEDIO_LLEGADAS = 500

In [59]:
metadata = MetaData()
msj_whatsapp_tbl = Table("msj_whatsapp", metadata, autoload_with=ENGINE, schema="notificaciones_wsp")

In [60]:
# 1️⃣ Función para consultar API con manejo de errores
async def extraer_datos(endpoint, fecha_ini, fecha_fin):
    url = f"http://10.1.0.103:9080/Utea/{endpoint}"
    params = {
        "pStrFecIni": fecha_ini,
        "pStrFecFin": fecha_fin,
    }
    timeout = httpx.Timeout(10.0)
    try:
        async with httpx.AsyncClient(timeout=timeout) as client:
            response = await client.get(url, params=params)
            if response.status_code == 200:
                now = datetime.now()
                formatted_now = now.strftime('%d/%m/%Y %H:%M:%S')
                print(formatted_now + " - GET: " + endpoint)
                return response.json()
            else:
                print(f"⚠️ Error {response.status_code} en {endpoint}")
                return None
    except httpx.RequestError as e:
        print(f"❌ Error de conexión con {endpoint}: {e}")
        return None

In [64]:
def es_hora_madrugada():
    ahora = datetime.now().time()
    return (ahora.hour >= 0 and ahora.hour < 7)

def calcular_horas_espera(df_tcb):
    df = df_tcb.copy()
    # elimina todos los registro sin datos de fechaDocum y HoraDocum
    #df = df.dropna(subset=['canero'])
    df = df[df['dateDocum'] != '0000-00-00']
    #extrae la hora para FECHA DE INICIO
    df['horaDocum'] = df['horaDocum'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_inicio'] = df['dateDocum'] + ' ' + df['horaDocum']
    # convierte la columna a tipo datetime
    df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'])
    #extraer la hora para FECHA DE FIN
    df['startTime'] = df['startTime'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_fin'] = df['startDate'] + ' ' + df['startTime']
    # convierte la columna a tipo datetime
    df['fecha_fin'] = pd.to_datetime(df['fecha_fin'])
    #calcula la diferencia
    df['espera'] = (df['fecha_fin'] - df['fecha_inicio']).dt.total_seconds() / 3600
    #retorn la media
    return df['espera'].mean()

def definir_trapiche(df):
    # extrae la hora
    df['solo_hora'] = df['hora'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_hora'] = df['fecha'] + ' ' + df['solo_hora']
    # convierte la columna a tipo datetime
    df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
    # obtiene la fecha y hora actual
    hora_actual = datetime.now()
    # calcula una hora antes
    una_hora_antes = hora_actual - timedelta(hours=1)
    # filtra los resgistros de la ultima hora
    df_ultima_hora = df[(df['fecha_hora'] >= una_hora_antes) & (df['fecha_hora'] <= hora_actual)]
    trapiches = list(set(df_ultima_hora['trapiche']))
    trapiches = [int(i) for i in trapiches]
    if len(trapiches) == 0:
        return 0
    elif 1 in trapiches and 2 in trapiches:
        return 3
    elif 1 in trapiches:
        return 1
    elif 2 in trapiches:
        return 2
    return 0

def get_horarios():
    try:
        df = pd.read_sql("SELECT * FROM datos_iag.horarios", ENGINE)
        return df
    except Exception as e:
        print("❌ Error al consultar PostgreSQL:", e)
        return pd.DataFrame()

def crear_alerta(datos):
    global VIAJES
    global PROMEDIO_LLEGADAS
    
    viajes_disponibles = datos['viajes_disponibles']
    promedio_llegada_viajes = datos['promedio_llegada_viajes']
    
    if viajes_disponibles > 200:
        if VIAJES <= 200:
            VIAJES = viajes_disponibles
            return True
    elif viajes_disponibles > 120:
        if VIAJES <= 120:
            VIAJES = viajes_disponibles
            return True
    elif viajes_disponibles >= 20 and viajes_disponibles <= 120:
        VIAJES = viajes_disponibles
        return False
    elif viajes_disponibles < 20:
        if VIAJES >= 20:
            VIAJES = viajes_disponibles
            return True
    elif promedio_llegada_viajes >= 8:
        if PROMEDIO_LLEGADAS < 8:
            PROMEDIO_LLEGADAS = promedio_llegada_viajes
            return True
        else:
            PROMEDIO_LLEGADAS = promedio_llegada_viajes
            return False
    else:
        return False
    return False

def crear_mensaje(datos):
    mensaje = f'''🔴⚠️ *ALERTA* ⚠️🔴
*📅 Hora act.:* {datos['hora']}
*⚙️ Trapiches:* {datos['trapiches_estado']}
*🚛 Viajes disponibles:* {round(datos['viajes_disponibles'],2)}
*🔢 Toneladas aprox.:* {round(datos['toneladas_aprox'],2)}
*⏱️ Promedio llegada vj.:* {round(datos['promedio_llegada_viajes'],2)}
*📈 Viajes estimados:* {round(datos['viajes_estimados'],2)}
*🕰️ Total horas abas.:* {round(datos['total_horas_abastecimiento'],2)}
*⏳ Tiempo espera:* {round(datos['tiempo_espera'],2)}
*🎋 Molienda actual:* {round(datos['molienda_actual'],2)}
*📅 Planificacion actual:* {round(datos['planificacion_actual'],2)}
*🔻 Diferencia actual:* {round(datos['diferencia_actual'],2)}
*🕒 Promedio horario:* {round(datos['promedio_horario'],2)}
*🏭 Molienda segun promedio:* {round(datos['molienda_segun_promedio'],2)}
*📊 Molienda segun estimacion:* {round(datos['molienda_segun_estimacion'],2)}'''

    mario_sanchez = '75380725'
    harold_pincker = '70249286'
    me = '78194371'

    data_mensajes = {
        "fecha_registro": [datetime.now(), datetime.now(), datetime.now()],
        "cod_canero": [0, 0, 0],
        "nombre_canero": ["Mario Sanchez Romero", "Harold Pinker", "Bismark Socompi"],
        "numero_cel": [mario_sanchez, harold_pincker, me],
        "mensaje": [mensaje, mensaje, mensaje],
        "enviado": [False, False, False],
        "fecha_enviado": [None, None, None]
    }
    
    df = pd.DataFrame(data_mensajes)
    
    try:
        with ENGINE.begin() as conn:
            conn.execute(msj_whatsapp_tbl.insert(), df.to_dict(orient='records'))
        print("✅ Se ha actualizado ")
    except Exception as e:
        print("❌ Error al insertar en tabla MSJ WHASTAPP", e)
    #return df

In [65]:
# 2️⃣ Simulación de función de cálculo
def calcular_datos(df_playa, df_trafCamBalanza, df_molienda):

    df_horarios = get_horarios()
    if df_horarios.empty:
        return {}
    df_molienda['hora2'] = df_molienda['hora2'].astype(int)
    df_res_molienda = pd.merge(df_molienda, df_horarios[['Hora', 'Orden_Hora']], left_on='hora2', right_on='Hora', how='left')
    
    #cantidad de paquetes
    #cantidad de caña disponible
    filtro = df_playa[(df_playa['status'] == 'PL') | (df_playa['status'] == 'IN')]
    cantidad_paquetes = filtro['cantPqt'].sum()
    cana_disponible = cantidad_paquetes * 45
    
    #promedio lleganda pq
    df_playa['dateCupo'] = pd.to_datetime(df_playa['dateCupo'])
    fecha_actual = pd.Timestamp('today').normalize()
    df_actual = df_playa[(df_playa['dateCupo'] == fecha_actual) & (df_playa['status'] != 'SL')].copy()
    df_actual['Hora_Entera'] = df_actual['horaDocum'].str[11:13].astype(int)
    max_hora_ent = df_actual['Hora_Entera'].max() - 3
    filtered_df = df_actual[df_actual['Hora_Entera'] >= max_hora_ent]
    sum_cant_pqt = filtered_df['cantPqt'].sum()
    promedio_llegada_pq = sum_cant_pqt / 3
    
    #trapiches
    # trapiche1    210 tn/ha    15 paquetes
    # trapiche2    690 tn/ha    49 paquetes
    
    #horas molienda
    horas_molienda_t1 = cantidad_paquetes / 5
    horas_molienda_t2 = cantidad_paquetes / 15
    horas_molienda_total = cantidad_paquetes / (5 + 15)
    
    #total paquetes resto dia
    total_paquetes_resto_dia_t1 = promedio_llegada_pq * horas_molienda_t1
    total_paquetes_resto_dia_t2 = promedio_llegada_pq * horas_molienda_t2
    total_paquetes_resto_dia_total = promedio_llegada_pq * horas_molienda_total
    
    #toneladas
    toneladas = df_molienda['netWeight'].sum() / 1000
    
    #planificacion actual
    planificacion_actual_t1 = df_res_molienda['Orden_Hora'].max() * 210
    planificacion_actual_t2 = df_res_molienda['Orden_Hora'].max() * 690
    planificacion_actual_total = df_res_molienda['Orden_Hora'].max() * (210 + 690)
    
    #diferencia actual
    diferencia_actual_t1 = toneladas - planificacion_actual_t1
    diferencia_actual_t2 = toneladas - planificacion_actual_t2
    diferencia_actual_total = toneladas - planificacion_actual_total
    
    #orden hora
    orden_hora = 24 - df_res_molienda['Orden_Hora'].max()
    
    #toneladas promedio
    toneladas_prom = (df_molienda['netWeight'].sum() / 1000) / (24 - orden_hora)

    #total horas
    total_horas_t1 = total_paquetes_resto_dia_t1 / (5) + horas_molienda_t1
    total_horas_t2 = total_paquetes_resto_dia_t2 / (15) + horas_molienda_t2
    total_horas_total = (total_paquetes_resto_dia_total / (5 + 15)) + horas_molienda_total

    #molienda segun promedio
    molienda_s_promedio = (toneladas_prom * orden_hora) + toneladas

    #molienda segun estimado
    molienda_s_estimado_t1 = toneladas + orden_hora * 210
    molienda_s_estimado_t2 = toneladas + orden_hora * 690
    molienda_s_estimado_total = toneladas + orden_hora * (210 + 690)

    #tiempo espera
    espera =  calcular_horas_espera(df_trafCamBalanza)
    trapiches = definir_trapiche(df_molienda)

    trapiches_estado = "01 y 02"
    viajes_disponibles = float(cantidad_paquetes)
    toneladas_aprox = float(cana_disponible)
    promedio_llegada_viajes = float(promedio_llegada_pq)
    viajes_estimados = float(total_paquetes_resto_dia_total)
    total_horas_abastecimiento = float(total_horas_total)
    tiempo_espera = float(espera)
    molienda_actual = float(toneladas)
    planificacion_actual = float(planificacion_actual_total)
    diferencia_actual = float(diferencia_actual_total)
    promedio_horario = float(toneladas_prom)
    molienda_segun_promedio = float(molienda_s_promedio)
    molienda_segun_estimacion = float(molienda_s_estimado_total)
    
    if trapiches == 0:
        trapiches_estado = "Detenidos"
        viajes_estimados = 0
        total_horas_abastecimiento = 0
        planificacion_actual = 0
        diferencia_actual = 0
        molienda_segun_estimacion = 0
    elif trapiches == 1:
        trapiches_estado = "solo 01"
        viajes_estimados = float(total_paquetes_resto_dia_t1)
        total_horas_abastecimiento = float(total_horas_t1)
        planificacion_actual = float(planificacion_actual_t1)
        diferencia_actual = float(diferencia_actual_t1)
        molienda_segun_estimacion = float(molienda_s_estimado_t1)
    elif trapiches == 2:
        trapiches_estado = "solo 02"
        viajes_estimados = float(total_paquetes_resto_dia_t2)
        total_horas_abastecimiento = float(total_horas_t2)
        planificacion_actual = float(planificacion_actual_t2)
        diferencia_actual = float(diferencia_actual_t2)
        molienda_segun_estimacion = float(molienda_s_estimado_t2)
    elif trapiches == 3:
        None
        #ya esta definido al inicio
    
    dict_datos = {
        "trapiches_estado": "01 y 02",
        "viajes_disponibles": float(cantidad_paquetes),
        "toneladas_aprox": float(cana_disponible),
        "promedio_llegada_viajes": float(promedio_llegada_pq),
        "viajes_estimados": float(total_paquetes_resto_dia_total),
        "total_horas_abastecimiento": float(total_horas_total),
        "tiempo_espera": float(espera),
        "molienda_actual": float(toneladas),
        "planificacion_actual": float(planificacion_actual_total),
        "diferencia_actual": float(diferencia_actual_total),
        "promedio_horario": float(toneladas_prom),
        "molienda_segun_promedio": float(molienda_s_promedio),
        "molienda_segun_estimacion": float(molienda_s_estimado_total)
    }
    return dict_datos

In [66]:
# 3️⃣ Función para guardar en PostgreSQL
def actualizar_reporte(datos):
    try:
        set_clause = ", ".join([f"{key} = :{key}" for key in datos.keys()])
        datos["id"] = 1  # condición WHERE id = 1
        query = text(f"UPDATE datos_iag.reporte SET {set_clause} WHERE id = :id")

        with ENGINE.begin() as conn:
            conn.execute(query, datos)
            print("✅ Registro actualizado correctamente (SQLAlchemy)")
    except Exception as e:
        print("❌ Error al actualizar con SQLAlchemy:", e)

In [ ]:
# 4️⃣ Función principal asíncrona
async def ciclo_principal():
    while True:
        hoy = datetime.now().strftime('%Y-%m-%d')
        ayer = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

        # Hacer todas las peticiones en paralelo
        tareas = [
            extraer_datos("ReportePlaya", ayer, hoy),
            extraer_datos("TrafCamBalanza", ayer, hoy),
            extraer_datos("Molienda", ayer if es_hora_madrugada() else hoy, ayer if es_hora_madrugada() else hoy)
        ]
        ReportePlaya, TrafCamBalanza, Molienda = await asyncio.gather(*tareas)
        # Validar que todas tengan datos antes de calcular
        if all([ReportePlaya, TrafCamBalanza, Molienda]):
            resultado = calcular_datos(
                pd.DataFrame(ReportePlaya), 
                pd.DataFrame(TrafCamBalanza), 
                pd.DataFrame(Molienda)
            )
            now = datetime.now()
            formatted_now = now.strftime('%H:%M:%S')
            resultado["hora"] = formatted_now
            
            actualizar_reporte(resultado)
            if crear_alerta(resultado):
                crear_mensaje(resultado)
                
        else:
            print("⚠️ No se pudieron obtener todos los datos. Se omite el cálculo.")
        await asyncio.sleep(300)  # espera 5 minutos

In [68]:
# 5️⃣ Ejecutar
if __name__ == "__main__":
    await ciclo_principal()

22/06/2025 18:02:14 - GET: Molienda
22/06/2025 18:02:14 - GET: ReportePlaya
22/06/2025 18:02:14 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLAlchemy)
22/06/2025 18:07:16 - GET: Molienda
22/06/2025 18:07:16 - GET: ReportePlaya
22/06/2025 18:07:16 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLAlchemy)
22/06/2025 18:12:18 - GET: Molienda
22/06/2025 18:12:18 - GET: ReportePlaya
22/06/2025 18:12:19 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLAlchemy)
22/06/2025 18:17:21 - GET: Molienda
22/06/2025 18:17:21 - GET: ReportePlaya
22/06/2025 18:17:21 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLAlchemy)
22/06/2025 18:22:23 - GET: Molienda
22/06/2025 18:22:23 - GET: ReportePlaya
22/06/2025 18:22:23 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLAlchemy)
22/06/2025 18:27:25 - GET: Molienda
22/06/2025 18:27:25 - GET: ReportePlaya
22/06/2025 18:27:26 - GET: TrafCamBalanza
✅ Registro actualizado correctamente (SQLA

CancelledError: 